# **Transfer Learning for Image-Based Malware Classification**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data='/content/drive/MyDrive/AI for cyber_sec/malimg_paper_dataset_imgs'

In [4]:
batches.class_indices

{'Adialer.C': 0,
 'Agent.FYI': 1,
 'Allaple.A': 2,
 'Allaple.L': 3,
 'Alueron.gen!J': 4,
 'Autorun.K': 5,
 'C2LOP.P': 6,
 'C2LOP.gen!g': 7,
 'Dialplatform.B': 8,
 'Dontovo.A': 9,
 'Fakerean': 10,
 'Instantaccess': 11,
 'Lolyda.AA1': 12,
 'Lolyda.AA2': 13,
 'Lolyda.AA3': 14,
 'Lolyda.AT': 15,
 'Malex.gen!J': 16,
 'Obfuscator.AD': 17,
 'Rbot!gen': 18,
 'Skintrim.N': 19,
 'Swizzor.gen!E': 20,
 'Swizzor.gen!I': 21,
 'VB.AT': 22,
 'Wintrim.BX': 23,
 'Yuner.A': 24}

In [3]:
from keras.preprocessing.image import ImageDataGenerator
batches = ImageDataGenerator().flow_from_directory(directory=data, target_size=(64,64), batch_size=10000)

Found 9339 images belonging to 25 classes.


In [5]:
imgs, labels = next(batches)

In [6]:
imgs.shape

(9339, 64, 64, 3)

In [7]:
labels.shape

(9339, 25)

In [9]:
classes = batches.class_indices.keys()

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs/255.,labels, test_size=0.3)

In [11]:
num_classes = 25

In [12]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [17]:
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input


In [37]:
#from tensorflow.keras.applications.resnet import 
from tensorflow.keras.applications import ResNet50

In [19]:
IMAGE_SIZE=[224,224]

In [20]:
inception=InceptionV3(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

87924736/87910968 [==============================] - 1s 0us/step


In [38]:
rsnet=ResNet50(input_shape=[64,64]+[3],weights='imagenet',include_top=False)

94781440/94765736 [==============================] - 1s 0us/step


In [21]:
#don't train existing weights
for layer in inception.layers:
  layer.trainable=False

In [39]:
#don't train existing weights
for layer in rsnet.layers:
  layer.trainable=False

In [22]:
x=Flatten()(inception.output)

In [40]:
x1=Flatten()(rsnet.output)

In [23]:
prediction=Dense(25,activation='softmax')(x)

In [41]:
prediction=Dense(25,activation='softmax')(x1)

In [25]:
model=Model(inputs=inception.input,outputs=prediction)

In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [42]:
model_rsnet=Model(inputs=rsnet.input,outputs=prediction)

In [45]:
model_rsnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 70, 70, 3)    0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 32, 32, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 32, 32, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

In [27]:
#tell the model what cost and optimization method to use
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [46]:
#tell the model what cost and optimization method to use
model_rsnet.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [29]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [35]:
X_train1=X_train.reshape()
   

TypeError: ignored

In [47]:
model_rsnet.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10
205/205 [==============================] - 138s 654ms/step - loss: 1.9798 - accuracy: 0.4239 - val_loss: 1.4981 - val_accuracy: 0.5382
Epoch 2/10
205/205 [==============================] - 134s 657ms/step - loss: 1.3058 - accuracy: 0.6036 - val_loss: 1.1482 - val_accuracy: 0.6392
Epoch 3/10
205/205 [==============================] - 134s 653ms/step - loss: 1.0539 - accuracy: 0.6688 - val_loss: 0.9628 - val_accuracy: 0.6799
Epoch 4/10
205/205 [==============================] - 134s 656ms/step - loss: 0.9224 - accuracy: 0.7006 - val_loss: 0.8996 - val_accuracy: 0.6995
Epoch 5/10
205/205 [==============================] - 131s 642ms/step - loss: 0.8385 - accuracy: 0.7220 - val_loss: 0.8084 - val_accuracy: 0.7120
Epoch 6/10
205/205 [==============================] - 135s 660ms/step - loss: 0.7836 - accuracy: 0.7401 - val_loss: 0.7498 - val_accuracy: 0.7313
Epoch 7/10
205/205 [==============================] - 136s 664ms/step - loss: 0.7462 - accuracy: 0.7393 - val_loss: 0.7301 -

In [49]:
from tensorflow.keras.applications import ResNet101

In [51]:
rsnet101=ResNet101(input_shape=[64,64]+[3],weights='imagenet',include_top=False)

In [52]:
#don't train existing weights
for layer in rsnet101.layers:
  layer.trainable=False

In [54]:
x2=Flatten()(rsnet101.output)

In [55]:
prediction2=Dense(25,activation='softmax')(x2)

In [57]:
model_rsnet101=Model(inputs=rsnet101.input,outputs=prediction2)

In [58]:
model_rsnet101.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 70, 70, 3)    0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 32, 32, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 32, 32, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

In [59]:
#tell the model what cost and optimization method to use
model_rsnet101.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [60]:
model_rsnet101.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10
205/205 [==============================] - 249s 1s/step - loss: 2.0552 - accuracy: 0.4195 - val_loss: 1.6814 - val_accuracy: 0.5243
Epoch 2/10
205/205 [==============================] - 248s 1s/step - loss: 1.4860 - accuracy: 0.5512 - val_loss: 1.3342 - val_accuracy: 0.5757
Epoch 3/10
205/205 [==============================] - 248s 1s/step - loss: 1.2458 - accuracy: 0.6124 - val_loss: 1.1513 - val_accuracy: 0.6317
Epoch 4/10
205/205 [==============================] - 248s 1s/step - loss: 1.1056 - accuracy: 0.6495 - val_loss: 1.0613 - val_accuracy: 0.6649
Epoch 5/10
205/205 [==============================] - 249s 1s/step - loss: 1.0092 - accuracy: 0.6729 - val_loss: 0.9788 - val_accuracy: 0.6959
Epoch 6/10
205/205 [==============================] - 252s 1s/step - loss: 0.9388 - accuracy: 0.6997 - val_loss: 0.9098 - val_accuracy: 0.7145
Epoch 7/10
205/205 [==============================] - 250s 1s/step - loss: 0.8861 - accuracy: 0.7086 - val_loss: 0.8651 - val_accuracy: 0.7163

In [66]:
from tensorflow.keras.applications import VGG19
#resnext.ResNeXt101(weights=None)

In [67]:
vgg_model=VGG19(input_shape=[64,64]+[3],weights='imagenet',include_top=False)

80150528/80134624 [==============================] - 1s 0us/step


In [68]:
#don't train existing weights
for layer in vgg_model.layers:
  layer.trainable=False

In [69]:
x3=Flatten()(vgg_model.output)

In [70]:
prediction3=Dense(25,activation='softmax')(x3)

In [71]:
model_vgg=Model(inputs=vgg_model.input,outputs=prediction3)

In [72]:
#tell the model what cost and optimization method to use
model_vgg.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [74]:
model_vgg.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10
205/205 [==============================] - 491s 2s/step - loss: 0.3125 - accuracy: 0.8914 - val_loss: 0.3346 - val_accuracy: 0.8919
Epoch 2/10
205/205 [==============================] - 492s 2s/step - loss: 0.2983 - accuracy: 0.8925 - val_loss: 0.3243 - val_accuracy: 0.9029
Epoch 3/10
205/205 [==============================] - 495s 2s/step - loss: 0.2932 - accuracy: 0.8948 - val_loss: 0.3185 - val_accuracy: 0.8911
Epoch 4/10
205/205 [==============================] - 495s 2s/step - loss: 0.2880 - accuracy: 0.8974 - val_loss: 0.3050 - val_accuracy: 0.9022
Epoch 5/10
205/205 [==============================] - 495s 2s/step - loss: 0.2794 - accuracy: 0.8993 - val_loss: 0.3182 - val_accuracy: 0.8954
Epoch 6/10
205/205 [==============================] - 495s 2s/step - loss: 0.2753 - accuracy: 0.9007 - val_loss: 0.2954 - val_accuracy: 0.9076
Epoch 7/10
205/205 [==============================] - 495s 2s/step - loss: 0.2667 - accuracy: 0.9026 - val_loss: 0.2920 - val_accuracy: 0.9083